<a href="https://colab.research.google.com/github/totiela/vk-ml-time-series-test-task/blob/main/notebooks/feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Импорт библиотек и загрузка данных

In [ ]:
# Установка неообходимых библиотек
!pip install -q tsfresh

In [6]:
# Импорт необходимых библиотек
import pandas as pd
import json
import numpy as np
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import impute
from sklearn.model_selection import train_test_split
import warnings

# Отключение всех предупреждений
warnings.filterwarnings("ignore")

In [3]:
# Клонируем репозитоиий
!git clone https://github.com/totiela/vk-ml-time-series-test-task.git
%cd vk-ml-time-series-test-task

Cloning into 'vk-ml-time-series-test-task'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 130 (delta 23), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (130/130), 47.81 MiB | 8.03 MiB/s, done.
Resolving deltas: 100% (23/23), done.
/content/vk-ml-time-series-test-task


In [4]:
# Загружаем тренировочный и тестовый датафреймы
train_path = "data/raw/train.parquet"
test_path = "data/raw/test.parquet"

train_df = pd.read_parquet(train_path)
test_df = pd.read_parquet(test_path)

# Генерация признаков

In [6]:
# Функция для извлечения признаков библиотекой tsfresh
def generate_features(df, column_id='id', column_dates='dates', column_values='values', date_format='%Y-%m-%d'):
    """
    Преобразует временные ряды в длинный формат, генерирует и обрабатывает признаки.

    Параметры:
    - df (pd.DataFrame): Исходный датафрейм с временными рядами.
    - column_id (str): Название колонки с уникальными идентификаторами.
    - column_dates (str): Название колонки с датами наблюдений.
    - column_values (str): Название колонки с соответствующими значениями.
    - date_format (str): Формат дат для преобразования в datetime, по умолчанию '%Y-%m-%d'.

    Возвращает:
    - pd.DataFrame: Датафрейм с сгенерированными признаками.
    """

    # Преобразование в длинный формат
    records = []
    for index, row in df.iterrows():
        for date, value in zip(row[column_dates], row[column_values]):
            records.append({column_id: row[column_id], 'date': date, 'value': value})

    long_df = pd.DataFrame(records)
    long_df['date'] = pd.to_datetime(long_df['date'], format=date_format)
    long_df = long_df.fillna(0)

    # Генерация признаков
    extracted_features = extract_features(
        long_df,
        column_id=column_id,
        column_sort='date',
        column_value='value'
    )

    # Иммутация пропущенных значений
    imputed_features = impute(extracted_features)

    # Приведение к упорядоченному индексу
    imputed_features = imputed_features.sort_index()

    return imputed_features

In [ ]:
# Применение функции к тренировочным и тестовым данным (выполняется долго)
imputed_features_test = generate_features(test_df)
imputed_features_train = generate_features(train_df)

In [5]:
# Всего получили более 700 признаков, но я отобрал топ-125 и топ-100 лучших признаков с помощью рекурсивного отбора значимости для модели catboost

# Загружаем данные из JSON-файла
with open("src/top_features.json", "r") as f:
    features = json.load(f)

# Доступ к спискам признаков
top_100_features = features["top_100"]
top_125_features = features["top_125"]

# Проверяем результат
print("Top 100 Features:", top_100_features)
print("Top 125 Features:", top_125_features)

Top 100 Features: ['value__variance_larger_than_standard_deviation', 'value__sum_values', 'value__abs_energy', 'value__mean_change', 'value__mean_second_derivative_central', 'value__median', 'value__mean', 'value__length', 'value__standard_deviation', 'value__variation_coefficient', 'value__variance', 'value__skewness', 'value__root_mean_square', 'value__longest_strike_below_mean', 'value__last_location_of_maximum', 'value__last_location_of_minimum', 'value__first_location_of_minimum', 'value__percentage_of_reoccurring_values_to_all_values', 'value__percentage_of_reoccurring_datapoints_to_all_datapoints', 'value__sum_of_reoccurring_values', 'value__sum_of_reoccurring_data_points', 'value__ratio_value_number_to_time_series_length', 'value__sample_entropy', 'value__quantile__q_0.1', 'value__quantile__q_0.3', 'value__quantile__q_0.4', 'value__quantile__q_0.6', 'value__quantile__q_0.7', 'value__quantile__q_0.8', 'value__quantile__q_0.9', 'value__autocorrelation__lag_2', 'value__autocorrela

In [25]:
# Функция для добавления ручных признаков

def extract_and_join_features(df, imputed_features):
    """
    Вычисляет дополнительные признаки для временных рядов и объединяет их с переданными сгенерированными признаками.

    Параметры:
    - df (pd.DataFrame): Исходный датафрейм с временными рядами.
    - imputed_features (pd.DataFrame): Датафрейм с ранее рассчитанными признаками.

    Возвращает:
    - pd.DataFrame: Датафрейм с объединенными ручными и сгенерированными признаками.

    Описание:
    - Вычисляет различные признаки на основе временных рядов, такие как:
        - dates_count: количество значений в каждой временной серии.
        - close_to_zero_diff: количество точек, близких к нулю по разнице значений.
        - quarterly_mean: среднее значение первых 4-х месяцев.
        - winter_percent и summer_percent: процент зимних и летних точек данных.
        - zero_crossings: количество переходов через ноль.
        - rolling_mean_X и rolling_std_X: скользящие средние и стандартное отклонение.
        - increasing_trend и decreasing_trend: количество точек с трендом на увеличение и уменьшение.
        - first_3_mean и last_3_mean: средние значения для первых и последних 3 значений ряда.
        - start_end_ratio: соотношение последних и первых 3 значений ряда.
        - seasonality_index: сезонный индекс, рассчитывающий сезонные изменения.

    - Удаляет ненужные признаки ('dates', 'values', 'values_diff', 'id') и объединяет результат с `imputed_features`.
    """

    # Количество значений в каждой временной серии
    df['dates_count'] = df['dates'].apply(lambda x: x.shape[0])

    # Производные значения ряда
    df['values_diff'] = df['values'].apply(lambda x: pd.Series(x).diff().dropna().round(4).to_numpy())

    # Количество точек, близких к нулю
    df['close_to_zero_diff'] = df['values_diff'].apply(lambda x: (np.abs(x) < 0.1).sum() if x.size > 0 else np.nan)

    # Среднее значение первых 4-х месяцев
    df['quarterly_mean'] = df['values'].apply(lambda x: np.mean(x[:4]) if len(x) >= 4 else np.nan)

    # Процент зимних и летних точек данных
    df['winter_percent'] = df['dates'].apply(lambda dates: np.mean([d.month in [12, 1, 2] for d in dates if pd.notnull(d)]) * 100)
    df['summer_percent'] = df['dates'].apply(lambda dates: np.mean([d.month in [6, 7, 8] for d in dates if pd.notnull(d)]) * 100)

    # Количество переходов через ноль
    df['zero_crossings'] = df['values'].apply(lambda x: np.sum(np.diff(np.sign(x)) != 0) if len(x) > 1 else 0)

    # Скользящие средние по значениям
    df['rolling_mean_3'] = df['values'].apply(lambda x: pd.Series(x).rolling(3).mean().iloc[-1] if len(x) >= 3 else np.nan)
    df['rolling_mean_6'] = df['values'].apply(lambda x: pd.Series(x).rolling(6).mean().iloc[-1] if len(x) >= 6 else np.nan)
    df['rolling_mean_12'] = df['values'].apply(lambda x: pd.Series(x).rolling(window=12).mean().iloc[-1] if len(x) >= 12 else np.nan)
    df['rolling_std_12'] = df['values'].apply(lambda x: pd.Series(x).rolling(window=12).std().iloc[-1] if len(x) >= 12 else np.nan)

    # Тренды на увеличение и уменьшение
    df['increasing_trend'] = df['values'].apply(lambda x: (np.diff(x) > 0).sum())
    df['decreasing_trend'] = df['values'].apply(lambda x: (np.diff(x) < 0).sum())

    # Среднее значение первых и последних 3 значений
    df['first_3_mean'] = df['values'].apply(lambda x: np.mean(x[:3]))
    df['last_3_mean'] = df['values'].apply(lambda x: np.mean(x[-3:]))

    # Соотношение между последними и первыми 3 значениями
    df['start_end_ratio'] = df['last_3_mean'] / (df['first_3_mean'] + 1e-5)

    # Сезонный индекс (для сезонных изменений)
    def seasonal_effect(values, dates):
        months_values = {m: [] for m in range(1, 13)}
        for val, date in zip(values, dates):
            months_values[date.month].append(val)
        return np.std([np.mean(months) for months in months_values.values() if months])

    df['seasonality_index'] = df.apply(lambda x: seasonal_effect(x['values'], x['dates']), axis=1)

    # Отбор рассчитанных признаков, исключая 'dates', 'values', 'id, 'values_diff и Join с imputed_features
    new_df = df.drop(['dates', 'values', 'values_diff', 'id'], axis=1)
    result_df = new_df.join(imputed_features)

    return result_df

In [26]:
# Получаем необходимые итоговые признаки для тренировочных и тестовых данных
test_116_features = extract_and_join_features(test_df, imputed_features_test[top_100_features])
test_141_features = extract_and_join_features(test_df, imputed_features_test[top_125_features])

train_116_features = extract_and_join_features(train_df, imputed_features_train[top_100_features])
train_141_features = extract_and_join_features(train_df, imputed_features_train[top_125_features])

In [ ]:
# Выгружаем итоговые датафреймы в формате .parquet
test_116_features.to_parquet('test_116_features.parquet')
test_141_features.to_parquet('test_141_features.parquet')

train_116_features.to_parquet('train_116_features.parquet')
train_141_features.to_parquet('test_141_features.parquet')